# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
import os

import pandas as pd
import numpy as np

In [ ]:
DATA_FOLDER = './data/'

In [205]:
ebola = pd.DataFrame(columns=['Month', 'Country', 'New', 'Deaths'])

def ebola_dirs(dirname, country, desc_col, totals_col, new_id, deaths_id):
    global ebola
    
    path = DATA_FOLDER + 'ebola/' + dirname
    for csv in os.listdir(path):
        frame = pd.read_csv(path + '/' + csv).fillna(0)
        
        month = parse(frame[frame.columns[0]][0]).month
        new = int(frame[frame[desc_col] == new_id][totals_col])
        deaths = int(frame[frame[desc_col] == deaths_id][totals_col])
        
        row = pd.DataFrame({'Month': month, 'Country': country, 'New': new, 'Deaths': deaths}, index=[0])
        ebola = ebola.append(row, ignore_index=True)

ebola_dirs('guinea_data/', 'Guinea', 'Description', 'Totals', 'New cases of confirmed', 'New deaths registered')
ebola_dirs('liberia_data/', 'Liberia', 'Variable', 'National', 'New case/s (confirmed)', 'Newly reported deaths')
ebola_dirs('sl_data/', 'SL', 'variable', 'National', 'new_confirmed', 'etc_new_deaths')

ebola["Deaths"] = pd.to_numeric(ebola["Deaths"])
ebola["New"] = pd.to_numeric(ebola["New"])
ebola.groupby(['Month', 'Country']).agg({'New': np.mean, 'Deaths': np.mean})

New     Deaths
Month Country                        
6     Liberia     2.142857   2.000000
7     Liberia     1.818182   4.272727
8     Guinea     12.400000   3.400000
      Liberia     5.444444  23.222222
      SL         18.550000   0.000000
9     Guinea     13.000000   3.562500
      Liberia     6.166667  36.041667
      SL         34.413793   0.275862
10    Guinea      6.000000  15.000000
      Liberia     1.360000  28.040000
      SL         56.714286   3.535714
11    Liberia     2.600000  13.466667
      SL         59.904762   0.571429
12    Liberia  1928.333333   0.000000
      SL         32.600000   2.200000

In [71]:
DATA_FOLDER = '' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
#We need to talk about what data we actually want to show in the end. But the way I wrote the code we can easily display
#the entries of some other row.

#Guinea
#list the c files for Guinea
filelist_Guinea = os.listdir('Data/ebola/guinea_data/');
#create a list of tables containing the data from the csv files, choose description as index
list_of_tables_Guinea = [pd.read_csv('Data/ebola/guinea_data/' + file,index_col = 'Description') for file in filelist_Guinea];
#concatenate list
Guinea = pd.concat(list_of_tables_Guinea)
#select Totals which gives table of indeces(Description) and totals
Guinea = Guinea['Totals']
#choose The desired descriptions
Guinea_new_cases = Guinea.loc['Total new cases registered so far'].astype(np.float32)
Guinea_deaths = Guinea.loc['New deaths registered'].astype(np.float32)

#Liberia
filelist_Liberia = os.listdir('Data/ebola/liberia_data/');
list_of_tables_Liberia = [pd.read_csv('Data/ebola/liberia_data/' + file,index_col = 'Variable') for file in filelist_Liberia];
Liberia = pd.concat(list_of_tables_Liberia)
Liberia = Liberia['National']
#The value returned here is too large. I didn't take the time to look into it.
Liberia_new_cases = Liberia.loc['New Case/s (Suspected)'].astype(np.float32)
Liberia_deaths = Liberia.loc['Newly reported deaths'].astype(np.float32)

#Sierra Leone
filelist_SL = os.listdir('Data/ebola/sl_data/');
list_of_tables_SL = [pd.read_csv('Data/ebola/sl_data/' + file,index_col = 'variable') for file in filelist_SL];
SL = pd.concat(list_of_tables_SL)
SL = SL['National']
SL_new_cases = SL.loc['new_confirmed'].astype(np.float32)
#For the deaths of Sierra Leone I know that I chose the wrong one.. We need to discuss what to do in this case
SL_deaths = SL.loc['death_confirmed'].astype(np.float32)

Ebola = pd.DataFrame({'Country':['Guinea', 'Liberia', 'Sierra-Leone'],
                     'Daily average of new cases':[Guinea_new_cases.mean(),Liberia_new_cases.mean(),SL_new_cases.mean()],'Daily average of deaths':[Guinea_deaths.mean(),Liberia_deaths.mean(),SL_deaths.mean()]})
Ebola

,Country,Daily average of deaths,Daily average of new cases
0,Guinea,4.045455,21.681818
1,Liberia,23.159090,207.546387
2,Sierra-Leone,773.343750,46.084209


## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here